<a href="https://colab.research.google.com/github/omkarpattnaik8080/Internshipproj02/blob/main/pwproj01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import random
from tqdm import tqdm
import numpy as np
import torch, torchvision
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.dataset import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import pandas as pd
from numpy import vstack
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from torch.optim import SGD
from torch.nn import BCELoss

In [ ]:
class PeopleDataset(T.utils.data.Dataset):
    def __init__(self, src_file, num_rows=None):
        df = pd.read_csv(src_file)
        df.drop(df.columns[[0]], axis=1, inplace=True)
        print(df.columns)
        df.Class = df.Class.astype('float64')
        y_tmp = df['Class'].values
        x_tmp = df.drop('Class', axis=1).values


        self.x_data = T.tensor(x_tmp,dtype=T.float64).to(device)
        self.y_data = T.tensor(y_tmp,dtype=T.float64).to(device)

        print(type(self.x_data))
        print(len(self.x_data))

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        if T.is_tensor(idx):
            idx = idx.tolist()
        preds = self.x_data[idx].type(T.FloatTensor)
        pol = self.y_data[idx].type(T.LongTensor)
        sample = [preds, pol]
        return sample

In [ ]:
class MLPUpdated(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(30, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32,16),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)


In [ ]:
fed_acc, fed_pre, fed_recall, fed_f1 = list(), list(), list(), list()
reg_acc, reg_pre, reg_recall, reg_f1 = list(), list(), list(), list()
split_acc, split_pre, split_recall, split_f1 = list(), list(), list(), list()


In [ ]:
# Can be changed to 12,16,24,32
num_clients = 32
# Change it to 3, 6, 10, 16
num_selected = 16
num_rounds = 50
epochs = 5
batch_size = 1024
device = "cpu"
device = torch.device(device)
fed_acc, fed_pre, fed_recall, fed_f1 = list(), list(), list(), list()


In [ ]:
def client_update(client_model, optimizer, train_loader, epoch=5):
    """
    This function updates/trains client model on client data
    """
    model.train()
    for e in range(epoch):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = client_model(data)
            binary_loss = torch.nn.BCEWithLogitsLoss()
            target = target.unsqueeze(1)
            target = target.float()
            loss = binary_loss(output, target)
            loss.backward()
            optimizer.step()
    return loss.item()


In [ ]:
def server_aggregate(global_model, client_models):
    """
    This function has aggregation method 'mean'
    """
    ### This will take simple mean of the weights of models ###
    global_dict = global_model.state_dict()
    for k in global_dict.keys():
        global_dict[k] = torch.stack([client_models[i].state_dict()[k].float() for i in range(len(client_models))], 0).mean(0)
    global_model.load_state_dict(global_dict)
    for model in client_models:
        model.load_state_dict(global_model.state_dict())

In [ ]:
def test(global_model, test_loader):
    """This function test the global model on test data and returns test loss and test accuracy """
    model.eval()
    test_loss = 0
    correct = 0
    actuals, predictions = list(), list()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = global_model(data)
            binary_loss = torch.nn.BCEWithLogitsLoss()
            target = target.unsqueeze(1)
            target = target.float()
            test_loss += binary_loss(output, target)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            actual = target.numpy()
            pr = output.detach().numpy()
            pr = pr.round()
            predictions.append(pr)
            actuals.append(actual)

    test_loss /= len(test_loader.dataset)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    # calculate precision
    prescision = precision_score(actuals, predictions)
    # calculate recall
    recall = recall_score(actuals, predictions)
    # calculate f1
    f1 = f1_score(actuals, predictions)
    fed_acc.append(acc)
    fed_pre.append(prescision)
    fed_recall.append(recall)
    fed_f1.append(f1)
    print()
    print(confusion_matrix(actuals, predictions))
    return test_loss, acc, prescision, recall, f1

In [ ]:
###########################################
#### Initializing models and optimizer  ####
############################################

#### global model ##########
# global_model =  MLPUpdated().cuda()
global_model = MLPUpdated().to(device)

############## client models ##############
# client_models = [ MLPUpdated().cuda() for _ in range(num_selected)]
client_models = [ MLPUpdated().to(device) for _ in range(num_selected)]
for model in client_models:
    model.load_state_dict(global_model.state_dict()) ### initial synchronizing with global model

############### optimizers ################
opt = [optim.SGD(model.parameters(), lr=0.01) for model in client_models]

In [ ]:
print(len(fed_acc))
print(len(fed_pre))
print(len(fed_recall))
print(len(fed_f1))


0
0
0
0


In [ ]:
###### List containing info about learning #########
losses_train = []
losses_test = []
acc_train = []
acc_test = []
# Runnining FL

import time
start_time = time.time()
for r in range(num_rounds):
    # select random clients
    client_idx = np.random.permutation(num_clients)[:num_selected]
    # client update
    loss = 0
    for i in tqdm(range(num_selected)):
        loss += client_update(client_models[i], opt[i], train_loader[client_idx[i]], epoch=epochs)

    losses_train.append(loss)
    # server aggregate
    server_aggregate(global_model, client_models)

    test_loss, acc, prescision, recall, f1= test(global_model, test_loader)
    losses_test.append(test_loss)
    acc_test.append(acc)
    print('%d-th round' % r)
    print('average train loss %0.3g | test loss %0.3g | test acc: %0.3f | test prescision: %0.3f | test recall: %0.3f | test f1: %0.3f' % (loss / num_selected, test_loss, acc, prescision, recall, f1))

print("--- %s seconds ---" % (time.time() - start_time))
# time[24]['fed'] = (time.time() - start_time)


  0%|          | 0/16 [00:00<?, ?it/s]


NameError: name 'train_loader' is not defined

In [3]:
print(len(fed_acc))
print(len(fed_pre))
print(len(fed_recall))
print(len(fed_f1))
# print(mp)
# print(len(mp))


NameError: name 'fed_acc' is not defined

In [ ]:
mp[num_selected] = {'fed_acc': fed_acc, 'fed_pre': fed_pre, 'fed_recall': fed_recall, 'fed_f1': fed_f1}
print(mp)
print(len(mp))

NameError: name 'mp' is not defined

In [2]:
reg_acc, reg_pre, reg_recall, reg_f1 = list(), list(), list(), list()
